In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
traindf = pd.read_csv('train_label.csv', dtype=str)

def append_ext(fn):
    return fn+".png"

traindf["ID"]=traindf["ID"].apply(append_ext)

In [6]:
# batch_size = 32

# datagen = ImageDataGenerator()
datagen = ImageDataGenerator(validation_split=0.25)
        
train_generator = datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = "train_image/",
    x_col = "ID",
    y_col = "Label",
    subset = "training",
#     batch_size = batch_size,
#     seed = 23,
#     shuffle = True,
    class_mode = "categorical",
    target_size = (512,512)
)

valid_generator = datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = "train_image/",
    x_col = "ID",
    y_col = "Label",
    subset = "validation",
#     batch_size = batch_size,
#     seed = 23,
#     shuffle = True,
    class_mode = "categorical",
    target_size = (512,512)
)

Found 873 validated image filenames belonging to 3 classes.
Found 291 validated image filenames belonging to 3 classes.


In [8]:
test_generator = datagen.flow_from_directory(
    "test_image", 
#     batch_size = batch_size, 
    class_mode=None, 
    shuffle=False,
    target_size = (512,512)
)

Found 292 images belonging to 1 classes.


In [13]:
model = Sequential()

model.add(Conv2D(input_shape=(512,512,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=3, activation="softmax"))

from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [14]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor = 'val_acc', verbose = 1,
                             save_best_only = True, save_weights_only = False, 
                             mode='auto')
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch = 100,
                           generator = train_generator, 
                           validation_data = valid_generator, 
                           validation_steps = 10,
                           epochs = 100,
                           callbacks=[checkpoint, early])

Epoch 1/100
 28/100 [=======>......................] - 1950s 70s/step - loss: 2258.9028 - accuracy: 0.4364 - val_loss: 0.8623 - val_accuracy: 0.6254


In [ ]:
plt.plot(hist.history["acc"])
plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 508, 508, 32)      896       
_________________________________________________________________
activation_5 (Activation)    (None, 508, 508, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 254, 254, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 252, 252, 32)      9248      
_________________________________________________________________
activation_6 (Activation)    (None, 252, 252, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 124, 124, 64)     

In [19]:
predicted = model.predict(
    test_generator,
#     batch_size=batch_size,
    verbose=0,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)

In [22]:
prediction = predicted.argmax(axis=1)

In [25]:
np.savetxt("predicted.csv", np.dstack((np.arange(prediction.size),prediction))[0],"%d,%d",header="ID,Label",comments="", )